In [3]:
import json
import web3

#from web3 import Web3
import os 
from web3 import Web3, HTTPProvider
from solc import compile_source
from web3.contract import ConciseContract

import pandas as pd

data = pd.read_csv("Llaves.csv") 

public_keys = data['Llave Publica']
private_keys = data['Llave Privada']

w3 = Web3(HTTPProvider("https://rinkeby.infura.io/v3/4c0ec7f1412a489d91e1934c66ebf5b1"))
hostname = "google.com" #example

## Transfer Ether

In [ ]:
acct = w3.eth.account.privateKeyToAccount(private_keys[0])
totalSupply = w3.eth.getBalance(public_keys[0])
print('Total Supply = '+str(totalSupply))
balance = 1
for address in public_keys[41:51]:
    
    signed_txn = w3.eth.account.signTransaction({
        'from': acct.address,
        'nonce': w3.eth.getTransactionCount(acct.address),
        'gas': 3000000,
        'to':address,
        'gasPrice': w3.toWei('1.4', 'gwei'),
        'value':w3.toWei(1,'ether')},
        private_keys[0])
    
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [ ]:
for i in range(1,51):
    acct = w3.eth.account.privateKeyToAccount(private_keys[i])
    address = public_keys[i+50]
    signed_txn = w3.eth.account.signTransaction({
        'from': acct.address,
        'nonce': w3.eth.getTransactionCount(acct.address),
        'gas': 3000000,
        'to':address,
        'gasPrice': w3.toWei('1.4', 'gwei'),
        'value':w3.toWei(0.5,'ether')},
        private_keys[i])
    
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

## Smart Contracts

In [5]:
## Coops

Coops_abi = open("./abi/Coops.json", "r")
c_values = json.load(Coops_abi)
Coops_abi.close()
coops = w3.eth.contract(
    address = '0x2c66b176962911D2ce40f5809cD52C73c8E78356',
    abi = c_values,
)

## Propietarios

Propietarios_abi = open("./abi/Propietarios.json", "r")
o_values = json.load(Propietarios_abi)
Propietarios_abi.close()
propietarios = w3.eth.contract(
    address = '0xD5fbF619121824aCB4e7aAf66A1d86947CE87f1B',
    abi = o_values,
)

## Proveedores

Proveedores_abi = open("./abi/Proveedores.json", "r")
v_values = json.load(Proveedores_abi)
Proveedores_abi.close()
proveedores = w3.eth.contract(
    address = '0x71eCEF369c041955C9993c635144a629c82CcD86',
    abi = v_values,
)

## Instaladores

Instaladores_abi = open("./abi/Instaladores.json", "r")
i_values = json.load(Instaladores_abi)
Instaladores_abi.close()
instaladores = w3.eth.contract(
    address = '0xC0ed1D311963EFc99418C0C84Ff2661CF199f9b0',
    abi = i_values
)

## Gestores

Gestores_abi = open("./abi/Gestores.json", "r")
m_values = json.load(Gestores_abi)
Gestores_abi.close()
gestores = w3.eth.contract(
    address = '0x23215E9FFaE52Eaf79c861d4F620A9748f3652C1',
    abi = m_values
)

## CF

CF_abi = open("./abi/CF.json", "r")
cf_values = json.load(CF_abi)
CF_abi.close()
cf = w3.eth.contract(
    address = '0x4Bac31B5056b1975D286d552F64F7962b8f2b2cc',
    abi = cf_values
)

## Proyectos

Proyectos_abi = open("./abi/Proyectos.json", "r")
p_values = json.load(Proyectos_abi)
Proyectos_abi.close()
proyectos = w3.eth.contract(
    address = '0x05c24902c594c255E4d2FA9B5D8ca7c1d44A7E85',
    abi = p_values
)

## Repartir Coops

In [16]:
i_addresses = 120
acct = w3.eth.account.privateKeyToAccount(private_keys[0])
response = os.system("ping -c 1 " + hostname)

while response != 0:
    #print('No Internet Connection')
    response = os.system("ping -c 1 " + hostname) 
    
balance = int(coops.call().balanceOf(public_keys[0])/(i_addresses)/1.0)
for i in range(1,i_addresses+1):
    
    response = os.system("ping -c 1 " + hostname)
    while response != 0:
        print('No Internet Connection')
        response = os.system("ping -c 1 " + hostname) 
    
    address = public_keys[i]
    i_balance = int(coops.call().balanceOf(public_keys[i]))
    if i_balance == 0: 
        construct_txn = coops.functions.transfer(address,balance).buildTransaction({
            'from': acct.address,
            'nonce': w3.eth.getTransactionCount(acct.address),
            'gas': 3000000,
            'gasPrice': w3.toWei('1.4', 'gwei')})
        signed = acct.signTransaction(construct_txn)

        tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
        # Wait for the transaction to be mined, and get the transaction receipt
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print('Balance of '+address+' = '+str(coops.call().balanceOf(address))+ '  Coops')

Balance of 0x6C740120A054cecF209BCaCF62fC38Cc64D98499 = 8297890  Coops
Balance of 0x520597cF4C6f9aed321a39a12529911350c7d0BF = 8297890  Coops
Balance of 0x6c2a6C2F7B3ABf37575a423E72d91C78F72B36E8 = 8297890  Coops
Balance of 0x29b93fdD9D9BE8c80d483aA94CA12496f09006C8 = 8297890  Coops
Balance of 0xF7ab85D5391651B239873fE0Cf26B6B2f0E52358 = 8297890  Coops
Balance of 0xe19e53113C2901eDec0Ff21a30Bcc28CFEB7c04B = 8297890  Coops
Balance of 0x2E3CaF2E65021aaA035c8ede24CCb361aC16659A = 8297890  Coops
Balance of 0x3f274122c33CED256d0F5D46f420ad6b92d4D9E5 = 8297890  Coops
Balance of 0xc30B42346A4d15AAdfA44cBf885E758dbe110b86 = 8297890  Coops
Balance of 0xa543D290F7B5FFCc94999935CF8E50f70a29C9bb = 8297890  Coops
Balance of 0x46D2b966Cf2195145aA3A9028aCDf0A1763E065F = 8297890  Coops
Balance of 0xAb7F3ccbBEb7d93605bDE9E0589F773FD4F21EC4 = 8297890  Coops
Balance of 0xbe9dAaA8007a325d21BA5104bCba75085e22b5E0 = 8297890  Coops
Balance of 0x13079510D41706bc7880d7C20Df1a5AaE3A26A25 = 8297890  Coops
Balanc

Balance of 0xeF65F2f6b637201bbC411232b2F8aF11F715dA54 = 8297890  Coops
Balance of 0x8871840eD37c5EaE1DFce560EF0b28821C916B94 = 8297890  Coops
Balance of 0x7fb65eba7B4904f1172fcB0FFE23d0b6e13a3DbF = 8297890  Coops
Balance of 0x88ea6915dDc5Dd6f34F4D9E05343291FeCA58448 = 8297890  Coops


## Devolver Coops

In [14]:
for i in range(1,len(public_keys)):
    address = public_keys[i]
    balance = coops.call().balanceOf(address)
    if balance > 0:
        try:
            acct = w3.eth.account.privateKeyToAccount(private_keys[i])
            construct_txn = coops.functions.transfer(public_keys[0],balance).buildTransaction({
                'from': acct.address,
                'nonce': w3.eth.getTransactionCount(acct.address),
                'gasPrice': w3.toWei('1.4', 'gwei')})

            signed = acct.signTransaction(construct_txn)

            tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
            # Wait for the transaction to be mined, and get the transaction receipt
            tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
            
        except:
            print ('Need to check '+ address)
        
    print('Balance of '+address+' = '+str(coops.call().balanceOf(address))+ '  Coops')



Balance of 0x6C740120A054cecF209BCaCF62fC38Cc64D98499 = 0  Coops
Balance of 0x520597cF4C6f9aed321a39a12529911350c7d0BF = 0  Coops
Balance of 0x6c2a6C2F7B3ABf37575a423E72d91C78F72B36E8 = 0  Coops
Balance of 0x29b93fdD9D9BE8c80d483aA94CA12496f09006C8 = 0  Coops
Balance of 0xF7ab85D5391651B239873fE0Cf26B6B2f0E52358 = 0  Coops
Balance of 0xe19e53113C2901eDec0Ff21a30Bcc28CFEB7c04B = 0  Coops
Balance of 0x2E3CaF2E65021aaA035c8ede24CCb361aC16659A = 0  Coops
Balance of 0x3f274122c33CED256d0F5D46f420ad6b92d4D9E5 = 0  Coops
Balance of 0xc30B42346A4d15AAdfA44cBf885E758dbe110b86 = 0  Coops
Balance of 0xa543D290F7B5FFCc94999935CF8E50f70a29C9bb = 0  Coops
Balance of 0x46D2b966Cf2195145aA3A9028aCDf0A1763E065F = 0  Coops
Balance of 0xAb7F3ccbBEb7d93605bDE9E0589F773FD4F21EC4 = 0  Coops
Balance of 0xbe9dAaA8007a325d21BA5104bCba75085e22b5E0 = 0  Coops
Balance of 0x13079510D41706bc7880d7C20Df1a5AaE3A26A25 = 0  Coops
Balance of 0xE793eF9f5D743F0Bb959144A34de76cD0b8eF6B4 = 0  Coops
Balance of 0xbd10542D544C

## Limpiar Deudas

In [12]:
for i in range(1,len(public_keys)):
    totaldebt = coops.call().getTotalDebt(public_keys[i])
    print('Debt of '+public_keys[i]+' = '+str(totaldebt)+ '  Coops')
    if totaldebt > 0:
        acct = w3.eth.account.privateKeyToAccount(private_keys[i])
        for address in public_keys:
            debt = coops.call().getDebt(acct.address,address)
            if debt > 0: 
                construct_txn = coops.functions.Pay(address,debt).buildTransaction({
                    'from': acct.address,
                    'nonce': w3.eth.getTransactionCount(acct.address),
                    'gas': 3000000,
                    'gasPrice': w3.toWei('1.4', 'gwei')})
                signed = acct.signTransaction(construct_txn)

                tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
                # Wait for the transaction to be mined, and get the transaction receipt
                tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
                print('Debt of '+acct.address+' = '+str(coops.call().getTotalDebt(acct.address))+ '  Coops')

Debt of 0x6C740120A054cecF209BCaCF62fC38Cc64D98499 = 15000  Coops
Debt of 0x6C740120A054cecF209BCaCF62fC38Cc64D98499 = 0  Coops
Debt of 0x520597cF4C6f9aed321a39a12529911350c7d0BF = 0  Coops
Debt of 0x6c2a6C2F7B3ABf37575a423E72d91C78F72B36E8 = 0  Coops
Debt of 0x29b93fdD9D9BE8c80d483aA94CA12496f09006C8 = 0  Coops
Debt of 0xF7ab85D5391651B239873fE0Cf26B6B2f0E52358 = 0  Coops
Debt of 0xe19e53113C2901eDec0Ff21a30Bcc28CFEB7c04B = 0  Coops
Debt of 0x2E3CaF2E65021aaA035c8ede24CCb361aC16659A = 0  Coops
Debt of 0x3f274122c33CED256d0F5D46f420ad6b92d4D9E5 = 0  Coops
Debt of 0xc30B42346A4d15AAdfA44cBf885E758dbe110b86 = 0  Coops
Debt of 0xa543D290F7B5FFCc94999935CF8E50f70a29C9bb = 0  Coops
Debt of 0x46D2b966Cf2195145aA3A9028aCDf0A1763E065F = 0  Coops
Debt of 0xAb7F3ccbBEb7d93605bDE9E0589F773FD4F21EC4 = 0  Coops
Debt of 0xbe9dAaA8007a325d21BA5104bCba75085e22b5E0 = 0  Coops
Debt of 0x13079510D41706bc7880d7C20Df1a5AaE3A26A25 = 0  Coops
Debt of 0xE793eF9f5D743F0Bb959144A34de76cD0b8eF6B4 = 0  Coops
Debt

## Display account; balances; debt

In [7]:
for i in range(1,len(public_keys)):
    address = public_keys[i]
    response = os.system("ping -c 1 " + hostname)
    while response != 0:
        print('No Internet Connection')
        response = os.system("ping -c 1 " + hostname) 
    balance = coops.call().balanceOf(address)
    totaldebt = coops.call().getTotalDebt(public_keys[i])
    print(public_keys[i]+'; '+str(balance)+ '; '+str(totaldebt))

0x6C740120A054cecF209BCaCF62fC38Cc64D98499; 8119290; 0
0x520597cF4C6f9aed321a39a12529911350c7d0BF; 8222990; 0
0x6c2a6C2F7B3ABf37575a423E72d91C78F72B36E8; 8297890; 0
0x29b93fdD9D9BE8c80d483aA94CA12496f09006C8; 8297890; 0
0xF7ab85D5391651B239873fE0Cf26B6B2f0E52358; 8297890; 0
0xe19e53113C2901eDec0Ff21a30Bcc28CFEB7c04B; 8297890; 0
0x2E3CaF2E65021aaA035c8ede24CCb361aC16659A; 8297890; 0
0x3f274122c33CED256d0F5D46f420ad6b92d4D9E5; 8297890; 0
0xc30B42346A4d15AAdfA44cBf885E758dbe110b86; 8297890; 0
0xa543D290F7B5FFCc94999935CF8E50f70a29C9bb; 8297890; 0
0x46D2b966Cf2195145aA3A9028aCDf0A1763E065F; 8297890; 0
0xAb7F3ccbBEb7d93605bDE9E0589F773FD4F21EC4; 8297890; 0
0xbe9dAaA8007a325d21BA5104bCba75085e22b5E0; 8297890; 0
0x13079510D41706bc7880d7C20Df1a5AaE3A26A25; 8282890; 0
0xE793eF9f5D743F0Bb959144A34de76cD0b8eF6B4; 8268190; 0
0xbd10542D544CC2bA88DBEaD8e387508DE1B1Fcc3; 8297890; 0
0x294566cbB8853B385e517e4902EDd193e9c5e05e; 8238190; 0
0x03F0c5E34D6cAc5aEA53128cedA7D1441fece1dC; 7500139; 0
0x3981aa54

## Detail Debt

In [97]:
debtor = 1
print(public_keys[debtor]+' totaldebt = ' + str(coops.call().getTotalDebt(public_keys[1])))
for i in range(0,len(public_keys)):
    print(public_keys[debtor]+' ows to '+public_keys[i]+' '+str(coops.call().getDebt(public_keys[1],public_keys[i]))+' Coops')

0x6C740120A054cecF209BCaCF62fC38Cc64D98499 totaldebt = 6969
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0xCBE861ab7726f2974289cE6b6aB7AD999D886E2F 0 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x6C740120A054cecF209BCaCF62fC38Cc64D98499 0 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x520597cF4C6f9aed321a39a12529911350c7d0BF 0 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x6c2a6C2F7B3ABf37575a423E72d91C78F72B36E8 0 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x29b93fdD9D9BE8c80d483aA94CA12496f09006C8 0 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0xF7ab85D5391651B239873fE0Cf26B6B2f0E52358 0 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0xe19e53113C2901eDec0Ff21a30Bcc28CFEB7c04B 0 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x2E3CaF2E65021aaA035c8ede24CCb361aC16659A 0 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x3f274122c33CED256d0F5D46f420ad6b92d4D9E5 0 Coops
0x6C740120A054cecF209BCaCF62fC3

0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x68038b6c0Ad8181045F14eE5726b4D41aD49FC56 134 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x40B9AB0d4ec7548934D7B823517cAD7E61413039 134 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x02e4F4b9a98Fd1f3f9FfB28C2996965127eFb779 134 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x9cd4E47cc7Ec65009DB7BEae5342be5E65A06Bb1 134 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x8EE3d7cAB013b9A9c516Ed6b2aD5BA9ed23727A5 134 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x04a970734e65f9c6f57CE484a33bc4BA7A2FA6A7 134 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x16ab7E894BE93b6adC09047C557c81fE63491bAC 134 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0x2879eC0BC6232e4bAC01D421aa62BE2d2cec3545 134 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0xd8A602136Ddd7462B7431f13d9720d2fbfC14Bff 134 Coops
0x6C740120A054cecF209BCaCF62fC38Cc64D98499 ows to 0xfd3Ead47b55f07af8EdBc

## Alcanzar Meta

In [68]:
acct = w3.eth.account.privateKeyToAccount(private_keys[5])
while cf.call().crowdsaleClosed() == True:
    pass

fundingGoal = cf.call().fundingGoal()
construct_txn = coops.functions.Fund(cf.address,fundingGoal).buildTransaction({
    'from': acct.address,
    'nonce': w3.eth.getTransactionCount(acct.address),
    'gas': 3000000,
    'gasPrice': w3.toWei('1', 'gwei')})

signed = acct.signTransaction(construct_txn)

tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)

# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print(tx_receipt['blockHash'].hex())
print('Gas used by Fund = '+str(tx_receipt.gasUsed))

KeyboardInterrupt: 

## delete demands

In [123]:
for i in range(1,len(public_keys)):
    address = public_keys[i]
    #acct = w3.eth.account.privateKeyToAccount(private_keys[i])
    if propietarios.call().isDemand(public_keys[i]) == True:
        acct = w3.eth.account.privateKeyToAccount(private_keys[i])
        construct_txn = propietarios.functions.deleteDemand(acct.address).buildTransaction({
            'from': acct.address,
            'nonce': w3.eth.getTransactionCount(acct.address),
            'gasPrice': w3.toWei('1', 'gwei')})
        signed = acct.signTransaction(construct_txn)
        tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
        print('Gas used by Delete Demand = '+str(tx_receipt.gasUsed))


## clear vendors

In [124]:
for i in range(1,len(public_keys)):
    #address = public_keys[i]
    if proveedores.call().isProvider(public_keys[i]) == True:
        acct = w3.eth.account.privateKeyToAccount(private_keys[i])
        construct_txn = proveedores.functions.deleteProvider(acct.address).buildTransaction({
            'from': acct.address,
            'nonce': w3.eth.getTransactionCount(acct.address),
            'gasPrice': w3.toWei('1', 'gwei')})
        signed = acct.signTransaction(construct_txn)
        tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
        print('Gas used by Delete Vendor = '+str(tx_receipt.gasUsed))


## clear installers

In [125]:

#clear demands

for i in range(1,len(public_keys)):
    address = public_keys[i]
    #acct = w3.eth.account.privateKeyToAccount(private_keys[i])
    if instaladores.call().isInstaller(public_keys[i]) == True:
        acct = w3.eth.account.privateKeyToAccount(private_keys[i])
        construct_txn = instaladores.functions.deleteInstaller(acct.address).buildTransaction({
            'from': acct.address,
            'nonce': w3.eth.getTransactionCount(acct.address),
            'gasPrice': w3.toWei('1', 'gwei')})
        signed = acct.signTransaction(construct_txn)
        tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
        print('Gas used by Delete Installer = '+str(tx_receipt.gasUsed))



## Clear Managers

In [126]:
for i in range(1,len(public_keys)):
    #address = public_keys[i]
    if gestores.call().isManager(public_keys[i]) == True:
        acct = w3.eth.account.privateKeyToAccount(private_keys[i])
        construct_txn = gestores.functions.deleteManager(acct.address).buildTransaction({
            'from': acct.address,
            'nonce': w3.eth.getTransactionCount(acct.address),
            'gasPrice': w3.toWei('1', 'gwei')})
        signed = acct.signTransaction(construct_txn)
        tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
        print('Gas used by Delete Manager = '+str(tx_receipt.gasUsed))

## Display Funfer/Fund

In [33]:
for i in range(1,len(public_keys)):
    address = public_keys[i]
    response = os.system("ping -c 1 " + hostname)
    while response != 0:
        print('No Internet Connection')
        response = os.system("ping -c 1 " + hostname) 
    balance = cf.call().balanceOf(address)
    print(public_keys[i]+'; '+str(balance))

0x6C740120A054cecF209BCaCF62fC38Cc64D98499; 0
0x520597cF4C6f9aed321a39a12529911350c7d0BF; 0
0x6c2a6C2F7B3ABf37575a423E72d91C78F72B36E8; 0
0x29b93fdD9D9BE8c80d483aA94CA12496f09006C8; 0
0xF7ab85D5391651B239873fE0Cf26B6B2f0E52358; 2331000
0xe19e53113C2901eDec0Ff21a30Bcc28CFEB7c04B; 0
0x2E3CaF2E65021aaA035c8ede24CCb361aC16659A; 0
0x3f274122c33CED256d0F5D46f420ad6b92d4D9E5; 0
0xc30B42346A4d15AAdfA44cBf885E758dbe110b86; 0
0xa543D290F7B5FFCc94999935CF8E50f70a29C9bb; 0
0x46D2b966Cf2195145aA3A9028aCDf0A1763E065F; 0
0xAb7F3ccbBEb7d93605bDE9E0589F773FD4F21EC4; 0
0xbe9dAaA8007a325d21BA5104bCba75085e22b5E0; 0
0x13079510D41706bc7880d7C20Df1a5AaE3A26A25; 0
0xE793eF9f5D743F0Bb959144A34de76cD0b8eF6B4; 0
0xbd10542D544CC2bA88DBEaD8e387508DE1B1Fcc3; 0
0x294566cbB8853B385e517e4902EDd193e9c5e05e; 0
0x03F0c5E34D6cAc5aEA53128cedA7D1441fece1dC; 0
0x3981aa541ECf2A9dB97e7e335Ebd7786E027B18B; 0
0xFca6cD79e30357Bee3fD194D07Aaec6aF435E3be; 0
0x098C9E49b954c3b80F77Bd8C9e1d3E77a07bbB67; 0
0x4c66E5e5AFc018E9b364Dd5279

In [ ]:
i_first = 25
i_last = 32
i_demands = i_last - i_first + 1

reference_price = int(2190000*0.3)
min_price = int(reference_price*0.1)

i_maxsteps = 10000
i_step = 1
i_matched = 0
#behaviour
while ((i_matched < i_demands) and (i_step <= i_maxsteps)):
    print('step '+str(i_step))
    i_matched = 0
    for i in range(i_first,i_last+1):
        b_matched = gestores.call().isManager(public_keys[i])
        if b_matched == False:
            i_matched += 1
        elif public_keys[i] == cf.call().vendor():
            pass
        elif ((b_matched == True) and ((i_step % 150) == 0)):
            #print('*Update Demand*')
            acct = w3.eth.account.privateKeyToAccount(private_keys[i])
            i_price_m = random.randint(min_price,reference_price)
            construct_txn = gestores.functions.updatePrice(i_price_m).buildTransaction({
                'from': acct.address,
                'nonce': w3.eth.getTransactionCount(acct.address),
                'gasPrice': w3.toWei('1', 'gwei')})
            signed = acct.signTransaction(construct_txn)
            tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
            tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print(i_matched)
    i_step += 1


## Simu Prop

In [ ]:
import json
import web3
import pdb
import os

#from web3 import Web3
from web3 import Web3, HTTPProvider
from solc import compile_source
from web3.contract import ConciseContract

import pandas as pd
import time
import random

data = pd.read_csv("Llaves.csv") 

public_keys = data['Llave Publica']
private_keys = data['Llave Privada']

w3 = Web3(HTTPProvider("https://rinkeby.infura.io/v3/4c0ec7f1412a489d91e1934c66ebf5b1"))

i_first = 1
i_last = 4
i_demands = i_last - i_first + 1

reference_price = 2190000
min_price = int(reference_price*0.4)

#create offers
for i in range(i_first,i_last+1):
    acct = w3.eth.account.privateKeyToAccount(private_keys[i])
    b_funding = bool(random.getrandbits(1))
    i_MaxPrice = reference_price #random.randint(min_price,reference_price)
    print('NewDemand('+str(b_funding)+', '+str(i_MaxPrice)+')')
    construct_txn = propietarios.functions.newDemand(b_funding,i_MaxPrice).buildTransaction({
        'from': acct.address,
        'nonce': w3.eth.getTransactionCount(acct.address),
        'gasPrice': w3.toWei('1', 'gwei')})

    signed = acct.signTransaction(construct_txn)

    tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)

    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print('Gas used by New Demand = '+str(tx_receipt.gasUsed))
